# Corruption analysis
Today is possible for us to readily and constantly check information online, over the years more and more people got access to mobile devices and this medium became the main for information exchange and gathering. We can imagine how this can influency society, but we're not going to study how people are interacting or being influenced, we're going to analyze the relations between the Internet users...

## About the data

We're going to use three datasets:
- Corruption Perception Index (CPI)
- Number Of Internet Users (# people with internet access)
- Number Of Journalists Killed

### Corruption Perception Index (CPI)
[Transparency International, CPI 2018](https://www.transparency.org/cpi2018)

More about the index (YouTube Video):

[![CPI 2018](https://i.ytimg.com/vi/OXApeTYRYNQ/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCNIW09Vs8woiyo5Y824JeMQarslQ)](https://www.youtube.com/watch?v=OXApeTYRYNQ)

**TODO**
Descriptions
- [ ] Intro description
- [ ] Dataset data explanation
  - [ ] Change CPI video thumbnail

Wrangling
- [x] Save new dataset
- [ ] Clean reshape code
- [ ] Create functions with reshaping options

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import operator
from functools import reduce


%matplotlib inline

df_internet = pd.read_csv('data/internet_users.csv')
df_killings = pd.read_csv('data/num_of_journalists_killed.csv')
df_cpi = pd.read_csv('data/corruption_perception_index_cpi.csv')

In [2]:
dfs = [df_internet, df_killings, df_cpi]
years = []

In [3]:
for d in dfs:
    print(d.shape)

(193, 29)
(100, 28)
(177, 7)


In [4]:
dfs[0].drop(dfs[0].columns.to_series()['1990':'2011'], axis=1, inplace=True)
dfs[0].drop('2017', axis=1, inplace=True)

In [5]:
dfs[1].drop(dfs[1].columns.to_series()['1992':'2011'], axis=1, inplace=True)
dfs[1].drop(['2017', '2018'], axis=1, inplace=True)

In [6]:
dfs[2].drop('2017', axis=1, inplace=True)

In [7]:
temp = dfs.copy()

In [8]:
temp[0].head()

,country,2012,2013,2014,2015,2016
0,Afghanistan,5.45,5.9,7.0,8.26,10.6
1,Albania,54.70,57.2,60.1,63.30,66.4
2,Algeria,18.20,22.5,29.5,38.20,42.9
3,Andorra,86.40,94.0,95.9,96.90,97.9
4,Angola,6.50,8.9,10.2,12.40,13.0


In [10]:
for i in range(0,len(temp)):
    temp[i]=temp[i].melt(id_vars='country', value_vars=temp[i].columns[1:7])
    temp[i].rename(columns={'variable': 'year'}, inplace=True)

In [11]:
temp[0].rename(columns={'value': 'internet_users'}, inplace=True)
temp[1].rename(columns={'value': 'journalists_killed'}, inplace=True)
temp[2].rename(columns={'value': 'cpi'}, inplace=True)

In [12]:
t1 = temp[0].merge(temp[1], on=['country', 'year'], how='inner')
t2 = t1.merge(temp[2], on=['country', 'year'], how='inner')

In [14]:
ind = list(t2.columns[0:2])
col = list(t2.columns[2:])

t3 = t2.pivot_table(index=ind, values=col)

t3.fillna(0, inplace=True)

In [19]:
group_killed = pd.DataFrame(t3.groupby('country')['journalists_killed'].sum())
none_killed = group_killed[group_killed['journalists_killed'] == 0].index

t3.drop(list(none_killed), axis=0, inplace=True)

In [27]:
# Save to CSV new dataset
t3.to_csv('data/cleaned_df_cpi-killings-internet.csv')